In [3]:
# Importando a biblioteca necessária
import pandas as pd

# Carregar o dataset com delimitador e codificação corretos
dataset = pd.read_csv(
    'DataSet.dsv', 
    delimiter=';', 
    encoding='latin1', 
    dtype={'CO_PA_AUTORIZ': str, 
           'CO_PA_CNSPAC': str,
           'CO_PA_PROC_ID': str,
           'CO_PA_MVM': str,
           'CO_PA_CNSMED': str},
    low_memory=False
)

# 1. Quantidade total e percentual de registros para pacientes identificados e não identificados (CO_PA_CNSPAC)
total_registros = len(dataset)
print(f"Quantidade de registros: {total_registros:,}".replace(',', '.'))
cnspac_identificado = dataset['CO_PA_CNSPAC'].notna().sum()
cnspac_nao_identificado = total_registros - cnspac_identificado
percentual_identificado = (cnspac_identificado / total_registros) * 100
percentual_nao_identificado = (cnspac_nao_identificado / total_registros) * 100

print("\nQuantidade e percentual de registros identificados e não identificados:")
print(f"Pacientes identificados (CO_PA_CNSPAC não nulo): {cnspac_identificado} registros ({percentual_identificado:.2f}%)")
print(f"Pacientes não identificados (CO_PA_CNSPAC nulo): {cnspac_nao_identificado} registros ({percentual_nao_identificado:.2f}%)")

# 2. Separar os registros de biópsia (0201010410) e anatomopatologia (0203020030)
biopsia = dataset[dataset['CO_PA_PROC_ID'] == '0201010410'].copy()  # Fazendo uma cópia explícita
anatomo = dataset[dataset['CO_PA_PROC_ID'] == '0203020030'].copy()  # Fazendo uma cópia explícita

# 3. Tratar os campos NU_PA_VALAPR para garantir que sejam numéricos
biopsia.loc[:, 'NU_PA_VALAPR'] = pd.to_numeric(biopsia['NU_PA_VALAPR'].str.replace(',', '.'), errors='coerce')
anatomo.loc[:, 'NU_PA_VALAPR'] = pd.to_numeric(anatomo['NU_PA_VALAPR'].str.replace(',', '.'), errors='coerce')

# 4. Agrupar e somar as quantidades e valores aprovados
biopsia_agrupada = biopsia.groupby(['CO_PA_CNSPAC', 'CO_PA_PROC_ID', 'CO_PA_MVM', 'CO_PA_CMP']).agg(
    NU_PA_QTDAPR=('NU_PA_QTDAPR', 'sum'),
    NU_PA_VALAPR=('NU_PA_VALAPR', 'sum')
).reset_index()

anatomo_agrupada = anatomo.groupby(['CO_PA_CNSPAC', 'CO_PA_PROC_ID', 'CO_PA_MVM', 'CO_PA_CMP']).agg(
    NU_PA_QTDAPR=('NU_PA_QTDAPR', 'sum'),
    NU_PA_VALAPR=('NU_PA_VALAPR', 'sum')
).reset_index()

# 5. Mesclar os dados de biópsia e anatomopatologia pelo paciente (CO_PA_CNSPAC)
pacientes_regulares = pd.merge(
    biopsia_agrupada, anatomo_agrupada, 
    on='CO_PA_CNSPAC', 
    suffixes=('_biopsia', '_anatomo')
)

# 6. Filtrar pacientes regulares:
# - Regular: 1 biópsia (NU_PA_QTDAPR_biopsia == 1) e 1 exame anatomopatológico (NU_PA_QTDAPR_anatomo == 1)
pacientes_regulares = pacientes_regulares[
    (pacientes_regulares['NU_PA_QTDAPR_biopsia'] == 1) & 
    (pacientes_regulares['NU_PA_QTDAPR_anatomo'] == 1)
]

# 7. Selecionar as colunas solicitadas
pacientes_regulares = pacientes_regulares[[
    'CO_PA_CNSPAC',
    'CO_PA_PROC_ID_biopsia', 'CO_PA_MVM_biopsia', 'CO_PA_CMP_biopsia', 
    'NU_PA_QTDAPR_biopsia', 'NU_PA_VALAPR_biopsia',
    'CO_PA_PROC_ID_anatomo', 'CO_PA_MVM_anatomo', 'CO_PA_CMP_anatomo', 
    'NU_PA_QTDAPR_anatomo', 'NU_PA_VALAPR_anatomo'
]]

# 8. Exportar o resultado para Excel
pacientes_regulares.to_excel('pacientes_regulares.xlsx', index=False)

print("\nArquivo 'pacientes_regulares.xlsx' exportado com sucesso!")

# -------------------------------------------
# Abaixo, agora vamos tratar os registros irregulares:

# 9. Filtrar pacientes irregulares:
# Irregular: Mais de 1 exame anatomopatológico (NU_PA_QTDAPR_anatomo > 1)
pacientes_irregulares = pd.merge(
    biopsia_agrupada, anatomo_agrupada, 
    on='CO_PA_CNSPAC', 
    suffixes=('_biopsia', '_anatomo')
)

# Filtrando os pacientes irregulares, ou seja, aqueles com mais de 1 exame anatomopatológico
pacientes_irregulares = pacientes_irregulares[
    (pacientes_irregulares['NU_PA_QTDAPR_anatomo'] > 1) | 
    (pacientes_irregulares['NU_PA_QTDAPR_biopsia'] > 1)
]

# 10. Agrupar os dados de biópsia e anatomopatologia
pacientes_irregulares_agrupados = pacientes_irregulares.groupby(
    ['CO_PA_CNSPAC', 'CO_PA_PROC_ID_biopsia', 'CO_PA_MVM_biopsia', 'CO_PA_CMP_biopsia', 
     'CO_PA_PROC_ID_anatomo', 'CO_PA_MVM_anatomo', 'CO_PA_CMP_anatomo']
).agg(
    NU_PA_QTDAPR_biopsia=('NU_PA_QTDAPR_biopsia', 'sum'),
    NU_PA_VALAPR_biopsia=('NU_PA_VALAPR_biopsia', 'sum'),
    NU_PA_QTDAPR_anatomo=('NU_PA_QTDAPR_anatomo', 'sum'),
    NU_PA_VALAPR_anatomo=('NU_PA_VALAPR_anatomo', 'sum')
).reset_index()

# 11. Exportar os dados irregulares para Excel
pacientes_irregulares_agrupados.to_excel('pacientes_irregulares.xlsx', index=False)

print("\nArquivo 'pacientes_irregulares.xlsx' exportado com sucesso!")


Quantidade de registros: 165.166

Quantidade e percentual de registros identificados e não identificados:
Pacientes identificados (CO_PA_CNSPAC não nulo): 163106 registros (98.75%)
Pacientes não identificados (CO_PA_CNSPAC nulo): 2060 registros (1.25%)

Arquivo 'pacientes_regulares.xlsx' exportado com sucesso!

Arquivo 'pacientes_irregulares.xlsx' exportado com sucesso!
